# CZ/CE 4041 Machine Learning
## Plant Seedling Classification [Kaggle] 
### Exploratory Data Analysis

### Team
* Dwivedee Lakshyajeet
* Gupta Jay
* Bansal Aditya
* Mantri Raghav
* Bhatia Ritik

> **Warning:** This notebook was created on the Kaggle platform where it fetches data from the Kaggle directories. It will not work by default on the Jupyter Notebook Platform.

In [1]:
# specify imports to be used in the notebook
import tensorflow as tf
import pandas as pd
import numpy as np
import os

In [2]:
# specify the training data directory
TRAIN_DATA_DIR = '../input/plant-seedlings-classification/train'

In [3]:
# specify the classes of the plants
species = ["Black-grass", "Charlock", "Cleavers", "Common Chickweed", "Common wheat", "Fat Hen",
                "Loose Silky-bent", "Maize", "Scentless Mayweed", "Shepherds Purse", "Small-flowered Cranesbill",
                "Sugar beet"]

### Defining the Model

In [9]:
# function that returns the model with custom layers appended
def define_model(width, height):
    
    # define the input to the model
    input_model = tf.keras.layers.Input(shape = (width, height, 3), name = 'image_input')
    
    # main model with Incpetion - ResNet - v2 layers
    # omit the top layers as we are adding custom layers
    # use transfer learning, with weights from Imagenet trained model
    main_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(include_top = False, weights = 'imagenet')(input_model)
    
    # flatten model to get appropriate dimensions
    flattened_model = tf.keras.layers.Flatten()(main_model)
    
    # add custom dropout and dense layers
    dropout_1 = tf.keras.layers.Dropout(0.5)(flattened_model)
    dense_1 = tf.keras.layers.Dense(128, activation = 'relu', activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_1)
    dropout_2 = tf.keras.layers.Dropout(0.5)(dense_1)
    
    # output of model
    output_model = tf.keras.layers.Dense(12, activation = "softmax", activity_regularizer=tf.keras.regularizers.l2(1e-5))(dropout_2)

    model = tf.keras.models.Model(input_model,  output_model)
    
    # use Adam optimizer with model
    optimizer = tf.keras.optimizers.Adam(lr = 5e-4, beta_1 = 0.9, beta_2 = 0.999)
    
    # use categorical crossentropy loss since classification task
    model.compile(loss="categorical_crossentropy", optimizer = optimizer, metrics = ["accuracy"])
    return model

In [5]:
# define appropriate callbacks
def training_callbacks():
    
    # save best model regularly
    save_best_model = tf.keras.callbacks.ModelCheckpoint(filepath = 'model.h5',
        monitor = 'loss', save_best_only = True, verbose = 1)
    
    # reduce learning rate when it stops decreasing
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'loss', factor = 0.4,
                              patience = 3, min_lr = 1e-10, verbose = 1, cooldown = 1)
    
    # stop training early if no further improvement
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'loss', min_delta = 1e-2, patience = 8, verbose = 1,
        mode = 'min', baseline = None, restore_best_weights = True
    )

    return [save_best_model, reduce_lr, early_stopping]

### Creating data generators, to feed into model

In [6]:
# create data generators
def data_generators():
    
    # apply random transformations on training data
    train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range = 360,
        shear_range = 0.3,
        zoom_range = 0.6,
        width_shift_range = 0.4,
        height_shift_range = 0.4,
        vertical_flip = True,
        horizontal_flip = True,
        validation_split = 1.0,
    )
    
    # define training data generator
    train_gen = train_data_generator.flow_from_directory(
        directory = TRAIN_DATA_DIR,
        target_size = (width, height),
        color_mode = 'rgb',
        class_mode = "categorical",
        batch_size = batch_size,
        subset = 'training',
    )
    
    # define test data generator
    test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator()
    test_gen = test_data_generator.flow_from_directory(
        directory = '../input/plant-seedlings-classification/',
        classes = ['test'],
        target_size = (width, height),
        batch_size = 1,
        color_mode = 'rgb',
        shuffle = False,
        class_mode = 'categorical')
    
    # define validation data generator
    validation_gen = train_data_generator.flow_from_directory(
        directory = TRAIN_DATA_DIR,
        color_mode = 'rgb',
        class_mode = "categorical",
        target_size = (width, height),
        batch_size = batch_size,
        subset = 'validation',
    )

    return train_gen, validation_gen, test_gen

### Training the model

In [7]:
# define training parameters
height = 299
width = 299
num_epochs = 100
batch_size = 16

In [ ]:
# define model and start training
model = define_model(width, height)
train_gen, validation_gen, test_gen = data_generators()

# the actual training
model.fit(
    train_gen,
    callbacks = training_callbacks()
    epochs = num_epochs,
    steps_per_epoch = train_gen.samples // batch_size,
    validation_data = validation_gen,
    validation_steps = validation_gen.samples // batch_size,
)

### Making predictions on test data

In [ ]:
# get model predictions
model_preds = model.predict(test_gen, steps = test_gen.samples)
classes = []

for data in range(0, model_preds.shape[0]):
    pred_index = model_preds[data, :].argmax(axis = -1)
    classes += [species[pred_index]]

In [ ]:
# generate submission csv
output_predictions = pd.DataFrame()
output_predictions['file'] = test_gen.filenames
output_predictions['file'] = output_predictions['file'].str.replace(r'test/', '')
output_predictions['species'] = classes
output_predictions.to_csv('output.csv', index = False)